In [1]:
import sys 
import json
import pandas as pd

sys.path.append("eval_utils")

from eval_utils.seat_v2_and_logprob import convert_tmps_to_sents, get_embeddings_and_probs, add_assos
from eval_utils.seat_v1 import get_embeddings, compute_effect_size
from eval_utils.evaluating_over_debiasing import add_predicts, get_vios_and_dif
from eval_utils.bias_utils.debiasing_effects_util import get_effect_size

eval_type = 'test'

BERT_BASE_UNCASED

Before Debiasing

In [5]:
tmp_path = '../data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents(tmp_path, eval_type,
                                                 simple = True)
sent2emb, model_type = get_embeddings_and_probs(corpus, tmps_type, temp_size, model_type = 'bert-base-uncased')
corpus = add_assos(corpus, tmps_type, temp_size, model_type = 'bert-base-uncased', sent2emb = sent2emb)

seat_effect_size, seat_p, log_effect_size, log_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))
print("For log probability score: effect size is {}, p value is {}".format(log_effect_size, log_p))

2021-09-12 19:19:01,539 : INFO : The debiasing effects evaluation type is test


load tmps from ../data/simple_patterns
--- 879 sentens to be processes ---


2021-09-12 19:19:04,022 : INFO : ****** The model is bert-base-uncased
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2021-09-12 19:19:08,872 : INFO : ******1 The type of used model is <class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>


--- get embeddings from TM and TAM---


  0%|                                                                                          | 0/543 [00:00<?, ?it/s]intial_eval_utils\bias_utils\utils_for_diverse_tmps.py:297: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  mask_index = (sents[sent_id] == tokenizer.mask_token_id).nonzero().flatten().tolist()[masked_id]
100%|████████████████████████████████████████████████████████████████████████████████| 543/543 [00:19<00:00, 27.77it/s]


--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:09<00:00, 34.57it/s]


For SEAT: effect size is 1.9427828883830491, p value is 0.0
For log probability score: effect size is 1.966382127046125, p value is 0.0


In [8]:
if eval_type == 'val':
    ori_seat_corpus_path = '../data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = '../data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = '../data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'bert-base-uncased')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
load model from bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


max_len evaluation: 128
--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:06<00:00, 35.05it/s]


For standard SEAT: effect size is 1.7004990198053145, p value is 0.0


In [13]:
data_without_preds = pd.read_csv('../data/2610_sents_for_evaluating_gender_loss.tsv', sep='\t', index_col = 0)
data = add_predicts(data_without_preds, model_type = 'bert-base-uncased')

violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 2610/2610 [03:03<00:00, 14.25it/s]


On UAL_Gender
{'error_num': 128, 'error_rate': 0.04904214559386973, 'IA': 2.51870070982319}


After Sent_debias

In [2]:
tmp_path = '../data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents(tmp_path, eval_type,
                                                 simple = True)
sent2emb, model_type = get_embeddings_and_probs(corpus, tmps_type, temp_size, model_type = 'bert-base-uncased', subspace_path = 'bert_save_embeddings/base_sent/gender_dir_0.05')
corpus = add_assos(corpus, tmps_type, temp_size, model_type = 'bert-base-uncased', sent2emb = sent2emb)

seat_effect_size, seat_p, log_effect_size, log_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))
print("For log probability score: effect size is {}, p value is {}".format(log_effect_size, log_p))

2021-09-12 19:32:34,785 : INFO : The debiasing effects evaluation type is test


load tmps from ../data/simple_patterns
* load gender dir from  bert_save_embeddings/base_sent/gender_dir_0.05
--- 879 sentens to be processes ---


2021-09-12 19:32:36,910 : INFO : ****** The model is bert-base-uncased
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2021-09-12 19:32:41,635 : INFO : ******1 The type of used model is <class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>


--- get embeddings from TM and TAM---


  0%|                                                                                          | 0/543 [00:00<?, ?it/s]intial_eval_utils\bias_utils\utils_for_diverse_tmps.py:297: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  mask_index = (sents[sent_id] == tokenizer.mask_token_id).nonzero().flatten().tolist()[masked_id]
100%|████████████████████████████████████████████████████████████████████████████████| 543/543 [00:39<00:00, 13.73it/s]


debiasing by sent_debias1...
--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:09<00:00, 34.12it/s]


debiasing by sent_debias2...
For SEAT: effect size is 1.3632698257145888, p value is 0.00377
For log probability score: effect size is 1.0976487652451454, p value is 0.02352


In [3]:
if eval_type == 'val':
    ori_seat_corpus_path = '../data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = '../data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = '../data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'bert-base-uncased', subspace_path = 'bert_save_embeddings/base_sent/gender_dir_0.05')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

* load gender dir from  bert_save_embeddings/base_sent/gender_dir_0.05
--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
load model from bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


max_len evaluation: 128
--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:06<00:00, 33.97it/s]


debiasing by sent_debias...
For standard SEAT: effect size is -0.09647898803104707, p value is 0.68141


In [2]:
data_without_preds = pd.read_csv('../data/2610_sents_for_evaluating_gender_loss.tsv', sep='\t', index_col = 0)
data = add_predicts(data_without_preds, model_type = 'bert-base-uncased', subspace_path = 'bert_save_embeddings/base_sent/gender_dir_0.05')

violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

* load gender dir from  bert_save_embeddings/base_sent/gender_dir_0.05


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                         | 0/2610 [00:00<?, ?it/s]C:\Hello_Mulin\My_study-base\experiments\phase2\Unimelb_fairness_project_phase2\bert\initial_evaluation\intial_eval_utils\evaluating_over_debiasing.py:87: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the fol

On UAL_Gender
{'error_num': 842, 'error_rate': 0.3226053639846743, 'IA': 0.09752957079234739}


BERT_LARGE_UNCASED

Before Debiasing

In [3]:
tmp_path = '../data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents(tmp_path, eval_type,
                                                 simple = True)
sent2emb, model_type = get_embeddings_and_probs(corpus, tmps_type, temp_size, model_type = 'bert-large-uncased')
corpus = add_assos(corpus, tmps_type, temp_size, model_type = 'bert-large-uncased', sent2emb = sent2emb)

seat_effect_size, seat_p, log_effect_size, log_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))
print("For log probability score: effect size is {}, p value is {}".format(log_effect_size, log_p))

2021-09-12 19:43:51,541 : INFO : The debiasing effects evaluation type is test


load tmps from ../data/simple_patterns
--- 879 sentens to be processes ---


2021-09-12 19:43:52,857 : INFO : ****** The model is bert-large-uncased
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2021-09-12 19:44:04,346 : INFO : ******1 The type of used model is <class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>


--- get embeddings from TM and TAM---


100%|████████████████████████████████████████████████████████████████████████████████| 543/543 [00:31<00:00, 17.38it/s]


--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:17<00:00, 19.18it/s]


For SEAT: effect size is 1.4932444391633548, p value is 0.00154
For log probability score: effect size is 1.9723877121303433, p value is 0.0


In [4]:
if eval_type == 'val':
    ori_seat_corpus_path = '../data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = '../data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = '../data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'bert-large-uncased')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
load model from bert-large-uncased


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


max_len evaluation: 128
--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:12<00:00, 19.12it/s]


For standard SEAT: effect size is 0.3350514372963267, p value is 0.15412


In [5]:
data_without_preds = pd.read_csv('../data/2610_sents_for_evaluating_gender_loss.tsv', sep='\t', index_col = 0)
data = add_predicts(data_without_preds, model_type = 'bert-large-uncased')

violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████| 2610/2610 [05:06<00:00,  8.51it/s]


On UAL_Gender
{'error_num': 36, 'error_rate': 0.013793103448275862, 'IA': 3.2428213802991674}


After Sent_debias

In [6]:
tmp_path = '../data/simple_patterns'

corpus, tmps_type, temp_size = convert_tmps_to_sents(tmp_path, eval_type,
                                                 simple = True)
sent2emb, model_type = get_embeddings_and_probs(corpus, tmps_type, temp_size, model_type = 'bert-large-uncased', subspace_path = 'bert_save_embeddings/large_sent/gender_dir_0.05')
corpus = add_assos(corpus, tmps_type, temp_size, model_type = 'bert-large-uncased', sent2emb = sent2emb)

seat_effect_size, seat_p, log_effect_size, log_p = get_effect_size(corpus)
print("For SEAT: effect size is {}, p value is {}".format(seat_effect_size, seat_p))
print("For log probability score: effect size is {}, p value is {}".format(log_effect_size, log_p))

2021-09-12 19:51:33,306 : INFO : The debiasing effects evaluation type is test


load tmps from ../data/simple_patterns
* load gender dir from  bert_save_embeddings/large_sent/gender_dir_0.05
--- 879 sentens to be processes ---


2021-09-12 19:51:34,566 : INFO : ****** The model is bert-large-uncased
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2021-09-12 19:51:44,546 : INFO : ******1 The type of used model is <class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>


--- get embeddings from TM and TAM---


100%|████████████████████████████████████████████████████████████████████████████████| 543/543 [01:07<00:00,  8.06it/s]


debiasing by sent_debias1...
--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:18<00:00, 18.61it/s]


debiasing by sent_debias2...
For SEAT: effect size is 0.3249787582763519, p value is 0.53781
For log probability score: effect size is 1.2563850790167035, p value is 0.00571


In [7]:
if eval_type == 'val':
    ori_seat_corpus_path = '../data/val_new_ori_seat_data.json'
elif eval_type == 'test':
    ori_seat_corpus_path = '../data/test_new_ori_seat_data.json'
else:
    assert eval_type == 'whole'
    ori_seat_corpus_path = '../data/new_ori_seat_data.json'

with open(ori_seat_corpus_path, 'r') as f:
    test = json.load(f)

sent2emb = get_embeddings(test, model_type = 'bert-large-uncased', subspace_path = 'bert_save_embeddings/large_sent/gender_dir_0.05')

seat_ez, p = compute_effect_size(test, sent2emb = sent2emb)

print("For standard SEAT: effect size is {}, p value is {}".format(seat_ez, p))

* load gender dir from  bert_save_embeddings/large_sent/gender_dir_0.05
--- 234 sentens to be processes ---
We will use the GPU: GeForce GTX 1650
load model from bert-large-uncased


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


max_len evaluation: 128
--- get embeddings from AM---


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:12<00:00, 19.42it/s]


debiasing by sent_debias...
For standard SEAT: effect size is 0.025817217813447542, p value is 0.91498


In [2]:
data_without_preds = pd.read_csv('../data/2610_sents_for_evaluating_gender_loss.tsv', sep='\t', index_col = 0)
data = add_predicts(data_without_preds, model_type = 'bert-large-uncased', subspace_path = 'bert_save_embeddings/large_sent/gender_dir_0.05')

violates, difs = get_vios_and_dif(data)

over_debias = {}

error_num = len(violates)
over_debias['error_num'] = error_num

error_rate = len(violates)/len(data)
over_debias['error_rate'] = error_rate

IA = sum(difs) / len(difs)
over_debias['IA'] = IA

print("On UAL_Gender")
print(over_debias)

* load gender dir from  bert_save_embeddings/large_sent/gender_dir_0.05


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                         | 0/2610 [00:00<?, ?it/s]C:\Hello_Mulin\My_study-base\experiments\phase2\Unimelb_fairness_project_phase2\bert\initial_evaluation\intial_eval_utils\evaluating_over_debiasing.py:87: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the fo

On UAL_Gender
{'error_num': 662, 'error_rate': 0.25363984674329504, 'IA': 0.20715517394418945}
